In [5]:
import pandas as pd
import numpy as np
# df1 = pd.read_csv("../datasets/startup_funding_1.csv", thousands=",")
# df1 = pd.read_csv("../datasets/startup_funding_2.csv")
df1 = pd.read_csv("../datasets/startup_investments/funds.csv")

print("Information of total number of non-empty columns")
print("-------------------------------------------------")
# print(df1.info(show_counts=True))

df1.info(show_counts=True)



Information of total number of non-empty columns
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1564 non-null   int64  
 1   fund_id               1564 non-null   int64  
 2   object_id             1564 non-null   object 
 3   name                  1564 non-null   object 
 4   funded_at             1449 non-null   object 
 5   raised_amount         1564 non-null   float64
 6   raised_currency_code  1564 non-null   object 
 7   source_url            1272 non-null   object 
 8   source_description    1218 non-null   object 
 9   created_at            1564 non-null   object 
 10  updated_at            1564 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 134.5+ KB


In [2]:
print("Frequency count of missing values")
df1.apply(lambda X:sum(X.isnull())) 

Frequency count of missing values


Sr No                   0
Date dd/mm/yyyy         0
Startup Name            0
Industry Vertical     171
SubVertical           936
City  Location        180
Investors Name         24
InvestmentnType         4
Amount in USD         964
Remarks              2625
dtype: int64

In [4]:
# Copying the df
df = df1.copy()
# df = df.dropna(axis=1)
df.rename(columns = {'Sr No': 's_no', 'Date dd/mm/yyyy':'date', 'Startup Name': 'startup_name', 'Industry Vertical': 'vertical', 'SubVertical': 'subvertical', 'City  Location': 'location', 'Investors Name': 'investors_name', 'InvestmentnType': 'investment_type', 'Amount in USD': 'amount_in_usd' }, inplace = True)
df = df.drop(labels = "Remarks", axis=1)
# df = df.drop(labels = "amount_in_usd", axis=0)
df = df[df['amount_in_usd'].notna()]
df.head()

print("Information of total number of non-empty columns")
print("-------------------------------------------------")
print(df.info(show_counts=True))


Information of total number of non-empty columns
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2080 entries, 0 to 3043
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   s_no             2080 non-null   int64 
 1   date             2080 non-null   object
 2   startup_name     2080 non-null   object
 3   vertical         1949 non-null   object
 4   subvertical      1426 non-null   object
 5   location         1945 non-null   object
 6   investors_name   2060 non-null   object
 7   investment_type  2077 non-null   object
 8   amount_in_usd    2080 non-null   object
dtypes: int64(1), object(8)
memory usage: 162.5+ KB
None


In [5]:
print("Frequency count of missing values")
df.apply(lambda X:sum(X.isnull())) 


Frequency count of missing values


s_no                 0
date                 0
startup_name         0
vertical           131
subvertical        654
location           135
investors_name      20
investment_type      3
amount_in_usd        0
dtype: int64

In [6]:
df

,s_no,date,startup_name,vertical,subvertical,location,investors_name,investment_type,amount_in_usd
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
3,4,02-01-2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"
...,...,...,...,...,...,...,...,...,...
3038,3039,28-01-2015,Grabhouse.com,NaN,NaN,NaN,"Kalaari Capital, Sequoia Capital",Private Equity,"25,00,000"
3039,3040,29-01-2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000"
3040,3041,29-01-2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000"
3041,3042,30-01-2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000"


In [111]:
#converting date to pd.datetime
# 05/072018
df['date'] = pd.to_datetime(df['date'],  errors = 'coerce', format="%d-%m-%Y")
df.head()
# df["date"].head(100)

,s_no,date,startup_name,vertical,subvertical,location,investors_name,investment_type,amount_in_usd
0,1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
2,3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
3,4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
4,5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"


In [133]:
# trying decision tree

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

data = df.copy()
y = data.amount_in_usd

features = ["location", "investors_name", "investment_type"]

X = data[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_test
# model = DecisionTreeClassifier(max_nodes = 3, random_state=1)

# model.fit(train_X, train_y)
# model.score(train_y, 
# val_predictions = model.predict(val_X)
# val_mae = mean_absolute_error(val_predictions, val_y)
# print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

,location,investors_name,investment_type
940,Mumbai,Atul Nishar,Seed Funding
385,Chennai,"Kris Gopalakrishnan, Pratithi Investment Trust...",Private Equity
102,Gurugram,Unnamed angel investors,Seed Funding
1512,Mumbai,"Sequoia Capital, Rohit Bansal, Kunal Bahl, San...",Seed Funding
2932,NaN,"Zodius Capital, Ru-Net, Kalaari Capital, Qualc...",Private Equity
...,...,...,...
319,Indore,InfoEdge,Seed/Angel Funding
332,Bengaluru,"Matrix Partners, Bhavish Aggarwal, Ankit Bhati...",Private Equity
147,Gurgaon,ZigExN,Series E
2130,New Delhi,"Ajeet Khurana Jinesh Shah, Vikram Chachra, Dr....",Seed Funding
